In [2]:
import numpy as np
import pandas as pd

In [2]:
import json

In [3]:
import os

In [4]:
import tensorflow as tf

In [5]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

import joblib

In [6]:
from bokeh.plotting import figure, show, output_file, save
from bokeh.io import output_notebook
from bokeh.layouts import row, gridplot, layout
from bokeh.palettes import d3
output_notebook()

Loading BokehJS ...

In [7]:
# a function to encode float values for serialized examples
def _float_feature_from_list_of_values(list_of_values):
    """Returns a float_list from a list of floats / doubles."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=list_of_values))

In [8]:
# the main source is the electricity dataset LD2011-2014 from UCI

In [9]:
# it resides in
dataset_path = '/home/developer/gcp/cbidmltsf/datasets/electricity'

In [10]:
os.listdir(dataset_path)

['LD2011_2014.txt',
 'separated_preprocessed',
 'separated_raw',
 'hourly_electricity_complete.pkl',
 'hourly_electricity.csv',
 'LD2011_2014.txt.zip',
 'hourly_electricity_filtered_academic_papers.pkl']

In [11]:
# 'LD2011_2014.txt'                                          source from UCI
# 'LD2011_2014.txt.zip'                                      source from UCI, compressed
# 'hourly_electricity.csv'                                   complete dataset in CSV
# 'hourly_electricity_complete.pkl'                          complete dataset in Pandas
# 'hourly_electricity_filtered_academic_papers.pkl'          filtered dataset for benchmarking
# 'separated_raw/'                                           pickles per customer, raw data
# 'separated_preprocessed/'                                  pickles per customer, outliers removed

In [12]:
# a SLDB is produced from separated time series (raw or preprocessed)

# SLDB contents are:
# TFRecord files for training
# TFRecord files for evaluation (if eval required)
# time series pickles for testing

In [13]:
# the structure of separated time series dataframes
some_customer = 'MT_320'
some_dataframe = pd.read_pickle('{}/separated_preprocessed/{}.pkl'.format(dataset_path,
                                                                          some_customer))
some_dataframe.head()

,power_usage,token_id,date,hours_from_start,days_from_start,hour_of_day,day_of_week,day_of_month,day_of_year,week_of_year,month_of_year
9099871,45.089186,320,2014-01-01 00:00:00,26304.0,1096,0,2,1,1,1,1
9099872,44.531773,320,2014-01-01 01:00:00,26305.0,1096,1,2,1,1,1,1
9099873,46.205407,320,2014-01-01 02:00:00,26306.0,1096,2,2,1,1,1,1
9099874,47.877648,320,2014-01-01 03:00:00,26307.0,1096,3,2,1,1,1,1
9099875,46.344760,320,2014-01-01 04:00:00,26308.0,1096,4,2,1,1,1,1


In [14]:
# constant values for positional encodings
hours_in_day = 24
days_in_week = 7
days_in_month = 30
days_in_year = 365

In [15]:
some_dataframe['hours_from_start'].min()

26304.0

In [16]:
# a constant to make sin/cos functions from hours_from_start (the 'age' covariate)
total_hours = some_dataframe['hours_from_start'].max()
total_hours

32303.0

In [17]:
# expand columns in test dataframe to review positional encodings

In [18]:
some_dataframe['sin_hour_day'] = np.sin(2*np.pi*some_dataframe.hour_of_day/hours_in_day)
some_dataframe['cos_hour_day'] = np.cos(2*np.pi*some_dataframe.hour_of_day/hours_in_day)
some_dataframe['sin_day_week'] = np.sin(2*np.pi*some_dataframe.day_of_week/days_in_week)
some_dataframe['cos_day_week'] = np.cos(2*np.pi*some_dataframe.day_of_week/days_in_week)
some_dataframe['sin_day_month'] = np.sin(2*np.pi*some_dataframe.day_of_month/days_in_month)
some_dataframe['cos_day_month'] = np.cos(2*np.pi*some_dataframe.day_of_month/days_in_month)
some_dataframe['sin_day_year'] = np.sin(2*np.pi*some_dataframe.day_of_year/days_in_year)
some_dataframe['cos_day_year'] = np.cos(2*np.pi*some_dataframe.day_of_year/days_in_year)

some_dataframe['sin_hours_from_start'] = np.sin(2*np.pi*some_dataframe.hours_from_start/total_hours)
some_dataframe['cos_hours_from_start'] = np.cos(2*np.pi*some_dataframe.hours_from_start/total_hours)

In [19]:
plots = dict()

In [24]:
# plot positional encodings for verification
label = 'positional'

plots[label] = figure(
    x_axis_type='datetime',
    plot_width=960,
    plot_height=400,
    title='Positional encodings for {}'.format(some_customer)
)

plots[label].grid.grid_line_alpha=0.3

plots[label].xaxis.axis_label = 'Date'
plots[label].yaxis.axis_label = 'Encoding value'

pos_encodings = [
    # 'sin_hour_day',
    # 'cos_hour_day',
    # 'sin_day_week',
    # 'cos_day_week',
    'sin_day_month',
    'cos_day_month',
    'sin_day_year',
    'cos_day_year',
    'sin_hours_from_start',
    'cos_hours_from_start'
]

for index, pos_encoding in enumerate(pos_encodings):
    plots[label].line(some_dataframe.date,
                      some_dataframe[pos_encoding],
                      # cycle the 10 values of the color palette
                      color=d3['Category10'][10][index%10],
                      legend_label=pos_encoding)

show(plots[label])

In [21]:
# split the time series in seen (train, eval) and unseen (test) data
# according to academic papers:

# 243 days on seen data, 7 days on unseen data 

# seen data:      '2014-01-01 00:00:00' to '2014-08-31 23:00:00', 243*24 = 5832 lectures

# train/eval split is 0.9/0.1, then

# train data:     '2014-01-01 00:00:00' to '2014-08-07 15:00:00', 5248 lectures
# eval data:      '2014-08-07 15:00:00' to '2014-08-31 23:00:00', 584 lectures

# unseen data:    '2014-09-01 00:00:00' to '2014-09-07 23:00:00', 7*24 = 168 lectures

# 243 weeks for seen data, 1 week for unseen data
no_lectures_seen_data = 243*24 # 5832

# seen data is divided as 90% for training and 10% for evaluation
train_eval_limit = 0.9

train_interval_end = int(no_lectures_seen_data*train_eval_limit) # 5248

In [22]:
# build sub-series to be persisted as serialized training examples

# dimensionality of the encoder input
m = 168

# dimensionality of the decoder output 
t = 168

In [23]:
# use 7D encoder (hour-day, day-week, day-year)

In [24]:
# columns to be included in the SLDB
sldb_columns = [
    'date',
    'token_id',
    'kw_scaled',
    'sin_hours_from_start',
    'cos_hours_from_start',
    'sin_hour_day',
    'cos_hour_day',
    'sin_day_week',
    'cos_day_week',
    # 'sin_day_month',
    # 'cos_day_month',
    # 'sin_day_year',
    # 'cos_day_year'    
]

In [25]:
sldb = {
    'ts': 'LD2011-2014_SEPARATED_MT_320-MT_330',
    'embedding': {
        'hourly': 168
    },
    'tau': {
        'hourly': 1
    },
    'no_targets': 168,
    'BSCTRFM': 1,
    'preprocessed': 1
}

In [26]:
sldb

{'ts': 'LD2011-2014_SEPARATED_MT_320-MT_330',
 'embedding': {'hourly': 168},
 'tau': {'hourly': 1},
 'no_targets': 168,
 'BSCTRFM': 1,
 'preprocessed': 1}

In [27]:
# a string with the basic specifications of the SLDB, as part of the SLDB identifier

# add the suffix '11D' to differentiate this SLDB from the original one, which is 9D

# add the suffix MMX to indicate the scaler used was MinMax
# add the suffix STD to indicate the scaler used was Standard

sldb_specs = 'BSCTRFM_{:03d}_{:03d}_07DB_MMX'.format(sldb['embedding']['hourly'], sldb['no_targets'])
sldb_specs

'BSCTRFM_168_168_07DB_MMX'

In [28]:
# get the time-based identifier for the SLDB
sldb_identifier = '{}_{}'.format(sldb['ts'], sldb_specs)
sldb_identifier

'LD2011-2014_SEPARATED_MT_320-MT_330_BSCTRFM_168_168_07DB_MMX'

In [29]:
sldb_dir = '/home/developer/gcp/cbidmltsf/sldbs/{}'.format(sldb_identifier)
sldb_dir

'/home/developer/gcp/cbidmltsf/sldbs/LD2011-2014_SEPARATED_MT_320-MT_330_BSCTRFM_168_168_07DB_MMX'

In [30]:
# make a directory for the complete SLDB
try:
    os.mkdir(sldb_dir)
    print('Directory {} was created.'.format(sldb_dir))
except FileExistsError:
    print('Error: directory {} already exists.'.format(sldb_dir))

Directory /home/developer/gcp/cbidmltsf/sldbs/LD2011-2014_SEPARATED_MT_320-MT_330_BSCTRFM_168_168_07DB_MMX was created.


In [31]:
# make a sub-directory for the training TFRecord files
try:
    os.mkdir('{}/train'.format(sldb_dir))
    print('Directory {}/train was created.'.format(sldb_dir))
except FileExistsError:
    print('Error: directory {}/train already exists.'.format(sldb_dir))

Directory /home/developer/gcp/cbidmltsf/sldbs/LD2011-2014_SEPARATED_MT_320-MT_330_BSCTRFM_168_168_07DB_MMX/train was created.


In [32]:
# make a sub-directory for the evaluation TFRecord files
try:
    os.mkdir('{}/eval'.format(sldb_dir))
    print('Directory {}/eval was created.'.format(sldb_dir))
except FileExistsError:
    print('Error: directory {}/eval already exists.'.format(sldb_dir))

Directory /home/developer/gcp/cbidmltsf/sldbs/LD2011-2014_SEPARATED_MT_320-MT_330_BSCTRFM_168_168_07DB_MMX/eval was created.


In [33]:
# make a sub-directory to persist time series pickles used for inference
try:
    os.mkdir('{}/test'.format(sldb_dir))
    print('Directory {}/test was created.'.format(sldb_dir))
except FileExistsError:
    print('Error: directory {}/test already exists.'.format(sldb_dir))

Directory /home/developer/gcp/cbidmltsf/sldbs/LD2011-2014_SEPARATED_MT_320-MT_330_BSCTRFM_168_168_07DB_MMX/test was created.


In [34]:
# get a path to the scalers sub-directory
scalers_dir = '{}/scalers'.format(sldb_dir)
scalers_dir

'/home/developer/gcp/cbidmltsf/sldbs/LD2011-2014_SEPARATED_MT_320-MT_330_BSCTRFM_168_168_07DB_MMX/scalers'

In [35]:
# make a sub-directory for the scalers
try:
    os.mkdir(scalers_dir)
    print('Directory {} was created.'.format(scalers_dir))
except FileExistsError:
    print('Error: directory {} already exists.'.format(scalers_dir))

Directory /home/developer/gcp/cbidmltsf/sldbs/LD2011-2014_SEPARATED_MT_320-MT_330_BSCTRFM_168_168_07DB_MMX/scalers was created.


In [36]:
encoder_input_columns = [
    'kw_scaled',
    'sin_hours_from_start',
    'cos_hours_from_start',
    'sin_hour_day',
    'cos_hour_day',
    'sin_day_week',
    'cos_day_week',
    # 'sin_day_month',
    # 'cos_day_month',
    # 'sin_day_year',
    # 'cos_day_year'
]

In [37]:
# both the encoder input and the decoder input use the same columns from the source sub_series dataframe
decoder_input_columns = encoder_input_columns

In [38]:
target_columns = ['kw_scaled']

In [39]:
id_columns = ['token_id']

In [49]:
# a dictionary to manage data per individual customer_id
data = dict()

In [50]:
# the number of cores available for training in Cloud TPU
num_cores = 8

In [51]:
start, end = 320, 330

In [52]:
token_ids = [token_id for token_id in np.arange(start, end + 1)]

In [53]:
customer_ids = ['MT_{:03d}'.format(token_id) for token_id in token_ids]

In [54]:
# are we training over raw data or preprocessed data?
state = 'preprocessed'

In [55]:
for customer_id in customer_ids:
    customer_data_path = '{}/separated_{}/{}.pkl'.format(dataset_path, state, customer_id)
    data[customer_id] = pd.read_pickle(customer_data_path)

In [56]:
data.keys()

dict_keys(['MT_320', 'MT_321', 'MT_322', 'MT_323', 'MT_324', 'MT_325', 'MT_326', 'MT_327', 'MT_328', 'MT_329', 'MT_330'])

### for SLDB generation, run this unified code!

In [57]:
for token_id in token_ids:
    
    # initialize the examples dictionary for each customer
    examples = {
        'train': [],
        'eval': [],
        # test dataset is not passed to SLDB
        # 'test': []
    }

    # get the customer identifier
    customer_id = 'MT_{:03d}'.format(token_id)
    customer_id
    print('Started processing for {}'.format(customer_id))

    # a temporary dataframe with data per customer_id to build the sub-series/examples
    # data_df = filtered_output[filtered_output['token_id'] == token_id].copy()
    
    # use now a reference to the dataframe in the data dictionary 
    data_df = data[customer_id]

    # expand with positional encodings
    data_df['sin_hours_from_start'] = np.sin(2*np.pi*data_df.hours_from_start/total_hours)
    data_df['cos_hours_from_start'] = np.cos(2*np.pi*data_df.hours_from_start/total_hours)
    data_df['sin_hour_day'] = np.sin(2*np.pi*data_df.hour_of_day/hours_in_day)
    data_df['cos_hour_day'] = np.cos(2*np.pi*data_df.hour_of_day/hours_in_day)
    data_df['sin_day_week'] = np.sin(2*np.pi*data_df.day_of_week/days_in_week)
    data_df['cos_day_week'] = np.cos(2*np.pi*data_df.day_of_week/days_in_week)
    # data_df['sin_day_month'] = np.sin(2*np.pi*data_df.day_of_month/days_in_month)
    # data_df['cos_day_month'] = np.cos(2*np.pi*data_df.day_of_month/days_in_month)
    # data_df['sin_day_year'] = np.sin(2*np.pi*data_df.day_of_year/days_in_year)
    # data_df['cos_day_year'] = np.cos(2*np.pi*data_df.day_of_year/days_in_year)

    # get a series for the power usage variable on the training dataset, to fit the scaler
    lectures_train_data = data_df['power_usage'][:train_interval_end]

    # fit a scaler only on train data
    # it is required to pass the power usage time series to a (?, 1) NumPy array
    lectures_train_data_array = np.array(lectures_train_data).reshape(-1, 1)
    
    # use MinMax scaler or Standard scaler

    # get MinMaxScaler on train data, store it in a dictionary
    scaler_type = 'min_max'
    scaler = MinMaxScaler()
    fitted_scaler = scaler.fit(lectures_train_data_array)
    print('Scaler {} generated on training data for {}'.format(scaler_type, customer_id))

    # persist the scaler
    scaler_filename = '{}/{}_{}.save'.format(scalers_dir, scaler_type, customer_id)
    joblib.dump(fitted_scaler, scaler_filename)
    print('Scaler {} persisted for {}'.format(scaler_type, customer_id))
    
    '''
    # get Standard on train data, store it in a dictionary
    scaler_type = 'standard'
    scaler = StandardScaler()
    fitted_scaler = scaler.fit(lectures_train_data_array)
    print('Scaler {} generated on training data for {}'.format(scaler_type, customer_id))

    # persist the scaler
    scaler_filename = '{}/{}_{}.save'.format(scalers_dir, scaler_type, customer_id)
    joblib.dump(fitted_scaler, scaler_filename)
    print('Scaler {} persisted for {}'.format(scaler_type, customer_id))
    '''
        
    # get an array from the variable time series (seen and unseen)
    all_data_variable_array = np.array(data_df.power_usage).reshape(-1, 1)

    # apply the scaler over all data (seen and unseen)
    # rescale, and squeeze to drop the extra dimension, then assign to the new column kw_scaled
    data_df['kw_scaled'] = np.squeeze(fitted_scaler.transform(all_data_variable_array))
    
    # at this moment, the individual time series is ready to be window-rolled to produce
    # sub-series/examples to serialize
    
    # BSCTRFM inference process is not direct, but iterative, therefoer
    # no TFRecord SLDB is required for test dataset,
    
    # persist only the time series corresponding to the inference interval as test dataset
    test_time_series = data_df[sldb_columns][no_lectures_seen_data - (m + t) + 1:]
    
    # path to persist the time series dataframe corresponding to test dataset
    path = '{}/test/{}.pkl'.format(sldb_dir, customer_id)
    
    test_time_series.to_pickle(path)
    print('Test dataset persisted as a time series pickle for {}'.format(customer_id))
    
    
    # get an iterable with all the possible sub-series for training examples
    for starting_point in np.arange(train_interval_end - (m + t) + 1):

        sub_series_df = data_df[sldb_columns][starting_point:starting_point + (m + t)]

        encoder_input_df = sub_series_df[encoder_input_columns][:m]
        decoder_input_df = sub_series_df[decoder_input_columns][m-1:m-1+t]
        target_df = sub_series_df[target_columns][m:m+t]
        id_df = sub_series_df[id_columns][:1]

        encoder_input_list = encoder_input_df.reset_index(drop=True).to_numpy().flatten().tolist()
        decoder_input_list = decoder_input_df.reset_index(drop=True).to_numpy().flatten().tolist()
        target_list = target_df.reset_index(drop=True).to_numpy().flatten().tolist()
        id_list = id_df.reset_index(drop=True).to_numpy().flatten().tolist()

        examples['train'].append(
            {
                'encoder_input': encoder_input_list,
                'decoder_input': decoder_input_list,
                'target': target_list,
                'id': id_list,
            }
        )

    print('{} processed. The number of examples in {} dataset is {}'.\
          format(customer_id, 'train', len(examples['train'])))


    # ToDo: remove evaluation step from Cloud TPU training and use all seen data for training stage
    build_eval_set = True

    if build_eval_set:

        # get an iterable with all the possible sub-series for evaluation examples
        for starting_point in np.arange(train_interval_end, no_lectures_seen_data - (m + t) + 1):

            sub_series_df = data_df[sldb_columns][starting_point:starting_point + (m + t)]

            encoder_input_df = sub_series_df[encoder_input_columns][:m]
            decoder_input_df = sub_series_df[decoder_input_columns][m-1:m-1+t]
            target_df = sub_series_df[target_columns][m:m+t]
            id_df = sub_series_df[id_columns][:1]

            encoder_input_list = encoder_input_df.reset_index(drop=True).to_numpy().flatten().tolist()
            decoder_input_list = decoder_input_df.reset_index(drop=True).to_numpy().flatten().tolist()
            target_list = target_df.reset_index(drop=True).to_numpy().flatten().tolist()
            id_list = id_df.reset_index(drop=True).to_numpy().flatten().tolist()

            examples['eval'].append(
                {
                    'encoder_input': encoder_input_list,
                    'decoder_input': decoder_input_list,
                    'target': target_list,
                    'id': id_list,
                }
            )

        print('{} processed. The number of examples in {} dataset is {}'.\
              format(customer_id, 'eval', len(examples['eval'])))


    # DO NOT PRODUCE A TEST DATASET FOR SLDB, AS INFERENCE PROCESS IS NOT DIRECT
    # (IT IS ITERATIVE OVER UNSEEN DATA TIME SERIES)

    # on each customer dataset, adjust the number of examples to the number of training cores
    for stage in ['train', 'eval']:
        # how many examples/rows must be removed from examples[stage] to comply with the number of cores
        examples_to_remove = len(examples[stage])%num_cores

        # remove the last 'examples_to_remove' examples from the dataset
        for _ in np.arange(examples_to_remove):
            examples[stage].pop(-1)

        print('For {} cores in Cloud TPU, the number of {} examples for {} was adjusted to {}'.\
             format(num_cores, stage, customer_id, len(examples[stage])))


    # serialize the rows in examples['train'] and, if present, examples['eval']
    # to avoid excesive memory consumption

    # write a TFRecord file for each consumer_id/stage
    for stage in ['train', 'eval']:
        # N_ROWS = sldb['stats'][stage]['n_rows']
        N_ROWS = len(examples[stage])
        filename = '{}/{}/{}.tfrecord'.format(sldb_dir, stage, customer_id)

        with tf.io.TFRecordWriter(filename) as writer:
            for row in np.arange(N_ROWS):

                example = tf.train.Example(
                    # features within the example
                    features=tf.train.Features(
                        # individual feature definition
                        feature={'encoder_input':
                                 _float_feature_from_list_of_values(
                                     examples[stage][row]['encoder_input']),
                                 'decoder_input':
                                 _float_feature_from_list_of_values(
                                     examples[stage][row]['decoder_input']),
                                 'target':
                                 _float_feature_from_list_of_values(
                                     examples[stage][row]['target']),
                                 'id':
                                 _float_feature_from_list_of_values(
                                     examples[stage][row]['id'])
                                 }
                    )
                )
                serialized_example = example.SerializeToString()
                writer.write(serialized_example)

            # report TFRecord file as completed
            print('Persisted {} TFRecord file for {}'.format(stage, customer_id))

Started processing for MT_320
Scaler min_max generated on training data for MT_320
Scaler min_max persisted for MT_320
Test dataset persisted as a time series pickle for MT_320
MT_320 processed. The number of examples in train dataset is 4913
MT_320 processed. The number of examples in eval dataset is 249
For 8 cores in Cloud TPU, the number of train examples for MT_320 was adjusted to 4912
For 8 cores in Cloud TPU, the number of eval examples for MT_320 was adjusted to 248
Persisted train TFRecord file for MT_320
Persisted eval TFRecord file for MT_320
Started processing for MT_321
Scaler min_max generated on training data for MT_321
Scaler min_max persisted for MT_321
Test dataset persisted as a time series pickle for MT_321
MT_321 processed. The number of examples in train dataset is 4913
MT_321 processed. The number of examples in eval dataset is 249
For 8 cores in Cloud TPU, the number of train examples for MT_321 was adjusted to 4912
For 8 cores in Cloud TPU, the number of eval e

In [58]:
# expand the sldb dictionary with final statistics
sldb['stats'] = {
    'train': {
        'n_rows': 54032,
    },
    'eval': {
        'n_rows': 2728
    }
}

In [59]:
sldb

{'ts': 'LD2011-2014_SEPARATED_MT_320-MT_330',
 'embedding': {'hourly': 168},
 'tau': {'hourly': 1},
 'no_targets': 168,
 'BSCTRFM': 1,
 'preprocessed': 1,
 'stats': {'train': {'n_rows': 54032}, 'eval': {'n_rows': 2728}}}

In [60]:
json_filename = '{}/sldb.json'.format(sldb_dir)

In [61]:
with open(json_filename, 'w') as filename:
    json.dump(sldb, filename, indent=4)